<a href="https://colab.research.google.com/github/cantaruttim/adai_college/blob/main/Modelagem_College.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta

In [2]:
def valida_arquivo(arquivo):
    extensao = os.path.splitext(arquivo)[1].lower()

    if extensao == ".csv":
        return pd.read_csv(f"{arquivo}", sep=";")
    elif extensao == ".xlsx":
        return pd.read_excel(f"{arquivo}")
    else:
        raise ValueError("Extensão de arquivo não suportada. Use .csv ou .xlsx")


# Gera a Base de alunos

In [26]:
def formatar_whatsapp(numero):
    if pd.isnull(numero):
        return numero
    numeros = re.sub(r"\D", "", numero)

    # Verifica se tem 11 dígitos (formato brasileiro com DDD + 9 dígitos)
    if len(numeros) == 11:
        ddd = numeros[:2]
        parte1 = numeros[2:7]
        parte2 = numeros[7:]
        return f"({ddd}) {parte1}-{parte2}"
    else:
        # Retorna como está se não tiver o padrão esperado
        return numero

def gerar_matricula(turma, telefone):
    turma_str = str(turma)
    primeiros_4 = turma_str[:4]
    ultimos_2 = turma_str[-2:]

    telefone_numeros = re.sub(r'\D', '', str(telefone))
    ultimos_4_telefone = telefone_numeros[-4:] if len(telefone_numeros) >= 4 else telefone_numeros

    resultado = primeiros_4 + ultimos_2 + ultimos_4_telefone

    return resultado


def gera_base_alunos(df):
    arquivo = "alunos_pb"

    df["Nome"] = df["Nome"].str.strip()
    df["Email"] = df["Email"].str.strip()

    df["Whatssap"] = df["Whatssap"].apply(formatar_whatsapp)
    df["Whatssap"] = df["Whatssap"].str.strip()

    # Aplicando a função a uma nova coluna
    df['Matricula'] = df.apply(
        lambda row:
                gerar_matricula(
                  row['Turma'],
                  row['Whatssap']
                ),
            axis=1
    )

    df["Matricula"] = df["Matricula"].astype(str)
    df["Matricula"] = pd.to_numeric(df["Matricula"], errors='coerce').fillna(0).astype(int)

    qtd_alunos_turma = (
        df.groupby("Turma")
          .size()
          .reset_index(name="qtdAlunosTurma")
    )

    df = df.merge(qtd_alunos_turma, on="Turma", how="left")

    df["qtdAlunosTurma"] = df["qtdAlunosTurma"].astype(int)

    df.to_excel(f"{arquivo}.xlsx", index=False)

df = pd.read_csv("/content/alunos.csv", sep=";")
gera_base_alunos(df)

In [ ]:
alunos = pd.read_excel("/content/alunos_pb.xlsx")
alunos

## Base para Análise das Frequências

# Gera a base Módulo

In [ ]:
meses_pt_en = {
    "jan": "Jan", "fev": "Feb", "mar": "Mar", "abr": "Apr", "mai": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug", "set": "Sep", "out": "Oct", "nov": "Nov", "dez": "Dec"
}

def data_valida(data_str, formato="%Y-%m-%d"):
    try:
        data = datetime.strptime(data_str, formato)
    except ValueError:
        return False  # Data inválida ou formato errado

    hoje = datetime.today()
    limite = data + timedelta(days=7)
    return hoje <= limite

def traduzir_mes(data_str):
    if not data_str:
        return None
    for pt, en in meses_pt_en.items():
        data_str = data_str.lower().replace(f"/{pt}", f"/{en}")
    return data_str

modulo = pd.read_csv("/content/Módulos.csv", sep=";")
modulo["status_data"] = modulo["DATA"].apply(lambda x: data_valida(x))

modulo["DATA"] = modulo["DATA"].fillna("").str.strip()
modulo["DATA"] = modulo["DATA"].apply(
    lambda x: f"{x}/2025" if x else None
)

modulo["DATA"] = modulo["DATA"].apply(traduzir_mes)
modulo["DATA"] = pd.to_datetime(modulo["DATA"], format="%d/%b/%Y", errors="coerce")

# Extrai o ano
modulo["Ano"] = modulo["DATA"].dt.year

modulo = modulo[["DATA", "Ano", "DIA DA SEMANA", "ATIVIDADE", "MÓDULO", "Semestre", "status_data"]]


modulo2 = (
    modulo.groupby("Semestre")
      .size()
      .reset_index(name="aulas_por_semestre")
)

# Cria a coluna 'ANOMES' no formato YYYYMM
modulo['ANOMES'] = modulo['DATA'].dt.strftime('%Y%m')

# Agrupa por 'ANOMES' e 'módulo' e conta a quantidade de aulas
# Corrigindo o nome da coluna de 'módulo' para 'MÓDULO'
aulas_por_anomes_modulo = modulo.groupby(['ANOMES', 'MÓDULO']).size().reset_index(name='quantidade_aulas')

modulo = modulo.merge(modulo2,
                      on="Semestre",
                      how="left"
)

modulo = modulo.merge(
    aulas_por_anomes_modulo,
    on=['ANOMES', 'MÓDULO'],
    how="left"
)

# Corrigindo a lista de nomes de colunas para incluir 'ANOMES'
modulo.columns = ["data",
                  "ano",
                  "dia_semana",
                  "atividade",
                  "módulo",
                  "semestre",
                  "status_data",
                  "ANOMES",
                  "aulas_por_semestre",
                  "quantidade_aulas"]

modulo

# Gera a base de Atividades

In [ ]:

def ler_arquivo_excel(file_path, sheet):
    try:
        df = pd.read_excel(
            file_path,
            sheet_name=sheet,
            engine="openpyxl"
        )
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {file_path}")
    except ValueError as e:
        print(f"Erro ao ler a planilha: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

    return df
atividades = ler_arquivo_excel("/content/atividades.xlsx", "Sheet1")
atividades